In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import pandas as pd
from package.utils import timer, load_data

In [3]:
INPUT_PATH = "s3://tli-crm-segmentation/data-zones/neutral/processed"
INPUT_FEATURE_PATH = f"{INPUT_PATH}/feature.csv"
INPUT_FAMILY_PATH = f"{INPUT_PATH}/family.csv"
INPUT_CUSTOMER_PATH = 's3://tli-crm-segmentation/data-zones/neutral/raw/tay_ds_customer_profile_202505091441.csv'

In [4]:
dtypes = {'party_rk':str}
feature_df = load_data(INPUT_FEATURE_PATH, dtype=dtypes)

function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/processed/feature.csv


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:286: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/processed/feature.csv
function: load_data successfully executed at 10.236475706100464s


In [4]:
@timer
def group_rider(data):
    proxy = data.copy()
    proxy['group_rider'] = None
    policy_count = proxy['policy_count_health'] + proxy['policy_count_ci']
    mask = policy_count == 0
    proxy.loc[mask, 'group_rider'] = 'rider'
    proxy['group_rider'] = proxy['group_rider'].fillna('norider')
    return proxy

In [5]:
@timer
def group_family(data, family_path):
    dtypes = {'party_rk':str}
    family_df = load_data(family_path, dtype=dtypes)
    mask = family_df['is_family']==1
    isfamily_list = set(family_df.loc[mask, 'party_rk'])

    proxy = data.copy()
    proxy['group_family'] = None
    mask = proxy['party_rk'].isin(isfamily_list)
    proxy.loc[mask, 'group_family'] = 'family'
    proxy['group_family'] = proxy['group_family'].fillna('nofamily')
    return proxy

In [6]:
@timer
def group_behavior(data):
    proxy = data.copy()
    proxy['group_behavior'] = 'other'
    proxy = group_1hold(data=proxy)
    proxy = group_0anp(data=proxy)
    proxy = group_99percentile_anp(data=proxy)
    return proxy

def group_99percentile_anp(data):
    proxy = data.copy()
    mask = proxy['total_premium'] > proxy['total_premium'].quantile(0.99)
    proxy.loc[mask, 'group_behavior'] = '99pct'
    return proxy

def group_0anp(data):
    proxy = data.copy()
    mask = proxy['total_premium'] == 0
    proxy.loc[mask, 'group_behavior'] = '0anp'
    return proxy

def group_1hold(data):
    proxy = data.copy()
    policy_count = proxy['policy_count_saving'] + proxy['policy_count_wholelife']
    mask = policy_count == 1
    proxy.loc[mask, 'group_behavior'] = '1hold'
    return proxy

In [29]:
def _get_lifestage(data):
    proxy = data.copy()
    proxy = proxy.loc[proxy['current_age']>=0, :]
    proxy['group_lifestage'] = None
    proxy.loc[proxy['current_age']>=0, 'group_lifestage'] = '0-20'
    proxy.loc[proxy['current_age']>=21, 'group_lifestage'] = '21-35'
    # proxy.loc[proxy['current_age']>=0, 'group_lifestage'] = '0-35'
    proxy.loc[proxy['current_age']>=36, 'group_lifestage'] = '36-50'
    proxy.loc[proxy['current_age']>=51, 'group_lifestage'] = '51-60'
    proxy.loc[proxy['current_age']>=61, 'group_lifestage'] = '61+'
    return proxy

@timer
def group_lifestage(data, customer_path):
    proxy = data.copy()
    dtypes = {'party_rk':str}
    customer_df = load_data(customer_path, dtype=dtypes)
    customer_df = _get_lifestage(data=customer_df)
    return pd.merge(left=proxy, right=customer_df[['party_rk', 'group_lifestage']], on='party_rk', how='inner')

In [21]:
proxy = feature_df.copy()

In [22]:
proxy = group_rider(data=proxy)
proxy = group_family(data=proxy, family_path=INPUT_FAMILY_PATH)
proxy = group_behavior(data=proxy)
proxy = group_lifestage(data=proxy, customer_path=INPUT_CUSTOMER_PATH)

function: group_rider is starting...
function: group_rider successfully executed at 0.4481046199798584s
function: group_family is starting...
function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/processed/family.csv
Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/processed/family.csv
function: load_data successfully executed at 2.2653636932373047s
function: group_family successfully executed at 4.505061626434326s
function: group_behavior is starting...
function: group_behavior successfully executed at 1.8545560836791992s
function: group_lifestage is starting...
function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/raw/tay_ds_customer_profile_202505091441.csv


/home/ec2-user/SageMaker/crm_pipeline/package/utils.py:119: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path, dtype=dtype)


Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/raw/tay_ds_customer_profile_202505091441.csv
function: load_data successfully executed at 32.61969614028931s
function: group_lifestage successfully executed at 56.13482093811035s


In [25]:
proxy['group_main'] = proxy['group_behavior'] + '_' + proxy['group_family'] + '_' + proxy['group_rider'] + '_' + proxy['group_lifestage']

In [45]:
proxy.set_index(conf.PARTY_ID_COLUMN)

,policy_count_saving,total_premium_saving,policy_count_wholelife,total_premium_wholelife,is_bene_parent,is_bene_child,is_bene_spouse,policy_count_health,total_premium_health,policy_count_ci,total_premium_ci,total_premium,bin_total_premium,group_rider,group_family,group_behavior,group_lifestage,group_main
party_rk,,,,,,,,,,,,,,,,,,
8000000052,1.0,0.0,0.0,0.0,0.0,0.500000,0.500000,3.0,19677.0,0.0,0.0,19677.0,2,norider,nofamily,1hold,61+,1hold_nofamily_norider_61+
8000000072,1.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0,rider,nofamily,0anp,61+,0anp_nofamily_rider_61+
8000000083,1.0,7000.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,7000.0,1,rider,nofamily,1hold,61+,1hold_nofamily_rider_61+
8000000085,1.0,0.0,0.0,0.0,1.0,0.000000,0.000000,3.0,12867.0,0.0,0.0,12867.0,1,norider,nofamily,1hold,36-50,1hold_nofamily_norider_36-50
8000000096,1.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0,rider,nofamily,0anp,21-35,0anp_nofamily_rider_21-35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8011617873,0.0,0.0,1.0,3228.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,3228.0,1,rider,nofamily,1hold,61+,1hold_nofamily_rider_61+
8011617875,0.0,0.0,1.0,10800.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,10800.0,1,rider,nofamily,1hold,51-60,1hold_nofamily_rider_51-60
8011617876,0.0,0.0,1.0,10764.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,10764.0,1,rider,nofamily,1hold,61+,1hold_nofamily_rider_61+


# dev script

In [29]:
from package.grouping.group_behavior import group_behavior
from package.grouping.group_family import group_family
from package.grouping.group_lifestage import group_lifestage
from package.grouping.group_rider import group_rider

In [6]:
proxy = feature_df.copy()
proxy = group_rider(data=proxy)
proxy = group_family(data=proxy, family_path=INPUT_FAMILY_PATH)
proxy = group_behavior(data=proxy)
proxy = group_lifestage(data=proxy, customer_path=INPUT_CUSTOMER_PATH)

function: group_rider is starting...
function: group_rider successfully executed at 0.49905824661254883s
function: group_family is starting...
function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/processed/family.csv
Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/processed/family.csv
function: load_data successfully executed at 2.520040988922119s
function: group_family successfully executed at 4.543239593505859s
function: group_behavior is starting...
function: group_behavior successfully executed at 1.9043402671813965s
function: group_lifestage is starting...
function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/raw/tay_ds_customer_profile_202505091441.csv


/home/ec2-user/SageMaker/crm_pipeline/package/utils.py:119: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path, dtype=dtype)


Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/raw/tay_ds_customer_profile_202505091441.csv
function: load_data successfully executed at 40.205538749694824s
function: group_lifestage successfully executed at 61.99632692337036s


## check rider

In [43]:
from package.grouping.group_rider import group_rider
proxy = group_rider(data=proxy)

function: group_rider is starting...
function: group_rider successfully executed at 0.7255234718322754s


In [44]:
mask = proxy['group_rider']=='norider'
proxy[mask].describe()[['policy_count_health', 'policy_count_ci']]

,policy_count_health,policy_count_ci
count,1871426.0,1871426.0
mean,0.0,0.0
std,0.0,0.0
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


In [45]:
mask = proxy['group_rider']=='rider'
((proxy[mask]['policy_count_health']+proxy[mask]['policy_count_ci'])==0).sum()

0

## check family

In [12]:
path = 's3://tli-crm-segmentation/data-zones/neutral/processed/family.csv'
dtypes = {'party_rk':str}
family_df = pd.read_csv(path, dtype=dtypes)

In [18]:
fam_list = family_df.loc[family_df['is_family']==1, 'party_rk']
mask = proxy['group_family'] == 'family'
mask &= ~proxy['party_rk'].isin(fam_list)
proxy[mask]

,party_rk,policy_count_saving,total_premium_saving,policy_count_wholelife,total_premium_wholelife,is_bene_parent,is_bene_child,is_bene_spouse,policy_count_health,total_premium_health,policy_count_ci,total_premium_ci,total_premium,bin_total_premium,group_rider,group_family,group_behavior,group_lifestage


In [20]:
fam_list = family_df.loc[family_df['is_family']==0, 'party_rk']
mask = proxy['group_family'] == 'nofamily'
mask &= ~proxy['party_rk'].isin(fam_list)
proxy[mask]

,party_rk,policy_count_saving,total_premium_saving,policy_count_wholelife,total_premium_wholelife,is_bene_parent,is_bene_child,is_bene_spouse,policy_count_health,total_premium_health,policy_count_ci,total_premium_ci,total_premium,bin_total_premium,group_rider,group_family,group_behavior,group_lifestage
